# Python实现英文新闻摘要自动提取

## 1.1 准备工作
`NLTK` 是建设一个Python程序与人类语言数据工作平台。它提供了易于使用的接口，超过50的语料库和词汇资源，如`WordNet`，连同一套文本处理库的分类、标记、标注、句法分析、语义推理的NLP库，和一个活跃的论坛。

要注意的是我们这次使用的一些词汇资源并不在原生的 NLTK 库中，需要我们另行下载。

在python 交互环境中，我们输入如下的代码来下载我们本次实现需要的资源。

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/yoran/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/yoran/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

之后我们在当前路径新建一个文件夹NewsSummary

先导入我们需要的包

In [1]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from string import punctuation
from heapq import nlargest

nltk.tokenize 是NLTK提供的分词工具包。所谓的分词 (tokenize) 实际就是把段落分成句子，把句子分成一个个单词的过程。我们导入的 sent_tokenize() 函数对应的是分段为句。 word_tokenize()函数对应的是分句为词。

stopwords 是一个列表，包含了英文中那些频繁出现的词，如am, is, are。

defaultdict 是一个带有默认值的字典容器。

puctuation 是一个列表，包含了英文中的标点和符号。

nlargest() 函数可以很快地求出一个容器中最大的n个数字。

至此我们完成了我们的准备工作。

## 1.2 思路解析
我们的基本思想很简单：拥有关键词最多的句子就是最重要的句子。我们把句子按照关键词数量的多少排序，取前n句，即可汇总成我们的摘要。

所以我们的工作可以分为如下步骤：

给在文章中出现的单词按照算法计算出重要性
按照句子中单词的重要性算出句子的总分
按照句子的总分给文章中的每个句子排序
取出前n个句子作为摘要
我们就按照这这个思路写我们的模块。

## 1.3 词频统计
首先我们先要统计出每个词在文章中出现的次数，在统计出次数之后，我们可以知道出现次数最多的词的出现次数 m 。我们把每个词 wi 出现的次数 mi 除以 m ，算出每个词的“重要系数”。
我们举个例子。 "I am a fool, big fool." 这句句子中

I   | am   | a  | fool | big
----|------|----|------|----
1   |1     | 1  |2     |1

出现最多的是“fool”这个单词，所以m是2。所有单词的频率都除以m出现最多的是 'fool' 这个单词，所以 m 是2。所有单词的频率都除以m，可以获得新的表

I   | am   | a  | fool | big
----|------|----|------|----
0.5 |0.5   |0.5 |1     |0.5

这张表显示的就是每个词的重要性。重要性高的词就是我们的关键词。

我们先定一些我们需要的常量：

In [2]:
stopwords = set(stopwords.words('english')+list(punctuation))
max_cut = 0.9
min_cut = 0.1

首先是我们的stopwords。

stopwords包含的是我们在日常生活中会遇到的出现频率很高的词，如do, I, am, is, are等等，这种词汇是不应该算是我们的关键字。同样的标点符号（punctuation）也不能被算作是关键字。

max_cut 变量限制了在文本中出现重要性过高的词。就像在跳水比赛中会去掉最高分和最低分一样。我们也需要去掉那些重要性过高和过低的词来提升算法的效果。

同理，min_cut 限制了出现频率过低的词。

In [3]:
'''
计算出每个词出现的频率
word_sent 是一个已经分好词的列表
返回一个词典freq[],
freq[w]代表了w出现的频率
'''
def compute_frequencise(word_sent):
    '''
    defaultdict和普通的dict
    的区别是它可以设置default值
    参数是int默认值是0
    '''
    freq = defaultdict(int)
    
    #统计每个词出现的频率
    for s in word_sent:
        for word in s:
            #注意stopwords
            if word not in stopwords:
                freq[word] += 1
    #得出最高出现频次m
    m = float(max(freq.values()))
    #所有单词的频次除以m
    for w in list(freq.keys()):
        freq[w] = freq[w]/m
        if freq[w] >= max_cut or freq[w] <= min_cut:
            del freq[w]
    # 最后返回的是
    # {key:单词, value: 重要性}
    return freq    

## 1.4获得摘要
现在每个单词（stopwords和出现频率异常的单词除外）都有了“重要性”这样一个量化描述的值。我们现在需要统计的是一个句子中单词的重要性。只需要把句子中每个单词的重要性叠加就行了。

In [4]:
def summarize(text,n):
    """
    用来总结的主要函数
    text是输入的文本
    n是摘要的句子个数
    返回包含摘要的列表
    """
    sents = sent_tokenize(text)
    assert n <= len(sents)
    
    # 然后再分词
    word_sent = [word_tokenize(s.lower()) for s in sents]
    
    # freq是一个词和词重要性的词典
    freq = compute_frequencise(word_sent)
    # ranking则是句子和句子重要性的词典
    ranking = defaultdict(int)
    for i,word in enumerate(word_sent):
        for w in word:
            if w in freq:
                ranking[i] += freq[w]
    sents_idx = rank(ranking,n)
    return [sents[j] for j in sents_idx]
        



'''
考虑到句子比较多的情况，
用遍历的方式找到最大的n个数比较慢
我们这里调用heapq中的函数
创建一个最小堆来完成这个功能
返回的是最小的n个数所在的位置
'''
def rank(ranking,n):
    return nlargest(n,ranking,key=ranking.get)

## 1.5运行程序
加入main（）

In [5]:
if __name__ == '__main__':
    with open('news.txt','r') as myfile:
        text = myfile.read().replace('\n','')
    res = summarize(text,2)
    for i in range(len(res)):
        print(res[i])

"Modern life is dramatically different to even 30 years ago," Prof Gray told Radio 4's Today programme, "people now drive to work and sit at work."
"The How Are You Quiz will help anyone who wants to take a few minutes to take stock and find out quickly where they can take a little action to make a big difference to their health."
